In [30]:
import pandas as pd
pd.options.display.max_rows=300

df_pat = pd.read_csv("/public/hanl/jupyter_dir/database/eicu/raw/patient.csv")
df_pat = df_pat[['patientunitstayid','hospitalid']]
df_pat.columns = ['stay_id','hospitalid']
df_hos = pd.read_csv("/public/hanl/jupyter_dir/database/eicu/raw/hospital.csv")
df_hos= df_hos[['hospitalid', 'region']]
df_demo = pd.merge(df_pat, df_hos, how="left",on="hospitalid")
df_demo.head(1)

,stay_id,hospitalid,region
0,147469,61,Midwest


In [ ]:
df_type = pd.read_csv("../../03.eICU_SAKI_trajCluster/df_mixAK_fea4_C3_eicu.csv")
df_type_filt = df_type.loc[:,["stay_id","groupHPD"]]
df_type_filt = df_type_filt.drop_duplicates()
all_lst = df_type_filt.stay_id.unique().tolist()
print("Number of included patients:",len(all_lst))
print(df_type_filt["groupHPD"].value_counts())

df_eicu = pd.read_csv("eicu_patient_middle_and_upper_right.csv")
right_up_lst = df_eicu.stay_id.unique()

df_type_filt["position"] = "left_down"
df_type_filt.loc[df_type_filt["stay_id"].isin(right_up_lst),"position"] = "right_up"

In [40]:
df_count_all = df.groupby(["position"]).agg({"stay_id":"count"}).reset_index()
df_count_all.columns = ["position", "count_all"]

# stat

In [44]:
df = pd.merge(df_demo, df_type_filt, how="inner",on="stay_id")
df_stat = df.groupby(["position","groupHPD"]).agg({"stay_id":"count"}).reset_index()
df_stat = pd.merge(df_stat,df_count_all, how="left",on="position")
df_stat["percent"] = round(df_stat["stay_id"] /df_stat["count_all"]*100,2)
df_stat

,position,groupHPD,stay_id,count_all,percent
0,left_down,1,308,679,45.36
1,left_down,2,275,679,40.50
2,left_down,3,96,679,14.14
3,right_up,1,393,1352,29.07
4,right_up,2,852,1352,63.02
5,right_up,3,107,1352,7.91


In [49]:
df_stat = df.groupby(["position","region"]).agg({"stay_id":"count"}).reset_index()
df_stat = pd.merge(df_stat,df_count_all, how="left",on="position")
df_stat["percent"] = round(df_stat["stay_id"] /df_stat["count_all"]*100,2)
df_stat.sort_values(["position","percent"])

,position,region,stay_id,count_all,percent
0,left_down,Midwest,130,679,19.15
1,left_down,Northeast,118,679,17.38
2,left_down,South,275,679,40.50
3,left_down,West,104,679,15.32
4,right_up,Midwest,494,1352,36.54
5,right_up,Northeast,48,1352,3.55
6,right_up,South,484,1352,35.80
7,right_up,West,241,1352,17.83


In [57]:
df_stat = df.groupby(["position","hospitalid"]).agg({"stay_id":"count"}).reset_index()
df_stat = pd.merge(df_stat,df_count_all, how="left",on="position")
df_stat["percent"] = round(df_stat["stay_id"] /df_stat["count_all"]*100,2)
df_stat = df_stat[df_stat["percent"] >2]
df_stat = df_stat.sort_values(["position","percent"],ascending=False)
df_stat[["position","hospitalid","percent"]] #[df_stat["position"] == "right_up"]

,position,hospitalid,percent
103,right_up,122,7.54
141,right_up,264,6.73
207,right_up,444,3.33
210,right_up,449,3.11
102,right_up,110,2.59
197,right_up,420,2.37
106,right_up,141,2.22
184,right_up,392,2.22
182,right_up,390,2.14
178,right_up,384,2.07
